<a href="https://colab.research.google.com/github/yerinsally/BDA/blob/main/%EC%A0%95%EC%98%88%EB%A6%B0_0402_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

titanic 데이터

- 타이타닉 데이터 전처리를 통해서 가장 높은 f1 스코어 만들기
- 교차검증 횟수는 5번 진행하되, 모델링은 자유롭게 하기

In [ ]:
import seaborn as sns
df = sns.load_dataset('titanic')
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, make_scorer

# 데이터 불러오기
titanic_df = sns.load_dataset('titanic')
titanic_df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [28]:
# 데이터 전처리
def preprocess_data(df):
    
    # 결측치 처리
    df['age'] = df['age'].fillna(df['age'].mean())
    df['fare'] = df['fare'].fillna(df['fare'].mean())
    df['embarked'] = df['embarked'].fillna('S')
    
    # 범주형 변수 전처리
    df['sex'] = df['sex'].map({'male': 0, 'female': 1}).astype(int)
    df = pd.concat([df, pd.get_dummies(df['embarked'], prefix='embarked')], axis=1)
    df = df.drop('embarked', axis=1)
    df = df.drop('embark_town', axis=1)
    df = df.drop('who', axis=1)
    df = df.drop('adult_male', axis=1)
    return df

titanic_df = preprocess_data(titanic_df)
titanic_df

,survived,pclass,sex,age,sibsp,parch,fare,class,deck,alive,alone,embarked_C,embarked_Q,embarked_S
0,0,3,0,22.000000,1,0,7.2500,Third,NaN,no,False,0,0,1
1,1,1,1,38.000000,1,0,71.2833,First,C,yes,False,1,0,0
2,1,3,1,26.000000,0,0,7.9250,Third,NaN,yes,True,0,0,1
3,1,1,1,35.000000,1,0,53.1000,First,C,yes,False,0,0,1
4,0,3,0,35.000000,0,0,8.0500,Third,NaN,no,True,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.000000,0,0,13.0000,Second,NaN,no,True,0,0,1
887,1,1,1,19.000000,0,0,30.0000,First,B,yes,True,0,0,1
888,0,3,1,29.699118,1,2,23.4500,Third,NaN,no,False,0,0,1
889,1,1,0,26.000000,0,0,30.0000,First,C,yes,True,1,0,0


In [29]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    891 non-null    int64   
 1   pclass      891 non-null    int64   
 2   sex         891 non-null    int64   
 3   age         891 non-null    float64 
 4   sibsp       891 non-null    int64   
 5   parch       891 non-null    int64   
 6   fare        891 non-null    float64 
 7   class       891 non-null    category
 8   deck        203 non-null    category
 9   alive       891 non-null    object  
 10  alone       891 non-null    bool    
 11  embarked_C  891 non-null    uint8   
 12  embarked_Q  891 non-null    uint8   
 13  embarked_S  891 non-null    uint8   
dtypes: bool(1), category(2), float64(2), int64(5), object(1), uint8(3)
memory usage: 61.5+ KB


In [30]:
titanic_df = pd.get_dummies(titanic_df, columns=['class','deck'])
titanic_df

,survived,pclass,sex,age,sibsp,parch,fare,alive,alone,embarked_C,...,class_First,class_Second,class_Third,deck_A,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G
0,0,3,0,22.000000,1,0,7.2500,no,False,0,...,0,0,1,0,0,0,0,0,0,0
1,1,1,1,38.000000,1,0,71.2833,yes,False,1,...,1,0,0,0,0,1,0,0,0,0
2,1,3,1,26.000000,0,0,7.9250,yes,True,0,...,0,0,1,0,0,0,0,0,0,0
3,1,1,1,35.000000,1,0,53.1000,yes,False,0,...,1,0,0,0,0,1,0,0,0,0
4,0,3,0,35.000000,0,0,8.0500,no,True,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.000000,0,0,13.0000,no,True,0,...,0,1,0,0,0,0,0,0,0,0
887,1,1,1,19.000000,0,0,30.0000,yes,True,0,...,1,0,0,0,1,0,0,0,0,0
888,0,3,1,29.699118,1,2,23.4500,no,False,0,...,0,0,1,0,0,0,0,0,0,0
889,1,1,0,26.000000,0,0,30.0000,yes,True,1,...,1,0,0,0,0,1,0,0,0,0


In [34]:
titanic_df['alive'] = titanic_df['alive'].replace({"no": 0, "yes": 1})
titanic_df['alone'] = titanic_df['alone'].astype(int)
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   survived      891 non-null    int64  
 1   pclass        891 non-null    int64  
 2   sex           891 non-null    int64  
 3   age           891 non-null    float64
 4   sibsp         891 non-null    int64  
 5   parch         891 non-null    int64  
 6   fare          891 non-null    float64
 7   alive         891 non-null    int64  
 8   alone         891 non-null    int64  
 9   embarked_C    891 non-null    uint8  
 10  embarked_Q    891 non-null    uint8  
 11  embarked_S    891 non-null    uint8  
 12  class_First   891 non-null    uint8  
 13  class_Second  891 non-null    uint8  
 14  class_Third   891 non-null    uint8  
 15  deck_A        891 non-null    uint8  
 16  deck_B        891 non-null    uint8  
 17  deck_C        891 non-null    uint8  
 18  deck_D        891 non-null    

In [35]:
# 학습 데이터, 테스트 데이터 분리
X_train = titanic_df.drop('survived', axis=1)
y_train = titanic_df['survived']
X_test = pd.DataFrame(columns=X_train.columns)

In [36]:
# 모델 학습 및 예측
models = [
    ('LogisticRegression', LogisticRegression()),
    ('KNN', KNeighborsClassifier()),
    ('DecisionTree', DecisionTreeClassifier()),
    ('RandomForest', RandomForestClassifier()),
    ('SVM', SVC()),
    ('GradientBoosting', GradientBoostingClassifier()),
    ('XGBoost', XGBClassifier()),
    ('LightGBM', LGBMClassifier())
]

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(f1_score)

In [37]:
best_f1_score = 0
best_model_name = None
for name, model in models:
    print('Model: ', name)
    param_grid = {}
    if name == 'LogisticRegression':
        param_grid = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}
    elif name == 'KNN':
        param_grid = {'n_neighbors': [3, 5, 7]}
    elif name == 'DecisionTree':
        param_grid = {'max_depth': [3, 5, 7]}
    elif name == 'RandomForest':
        param_grid = {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7]}
        cv_result = GridSearchCV(model, param_grid=param_grid, scoring=scorer, cv=kfold, n_jobs=-1)
        cv_result.fit(X_train, y_train)
        
        y_pred = cv_result.predict(X_train)
        f1 = f1_score(y_train, y_pred)
        print('Best parameters:', cv_result.best_params_)
        print('Best F1 score:', f1)
        
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model_name = name
            best_model = cv_result.best_estimator_

print('\nBest model:', best_model_name)
print('Best F1 score:', best_f1_score)

Model:  LogisticRegression
Model:  KNN
Model:  DecisionTree
Model:  RandomForest
Best parameters: {'max_depth': 3, 'n_estimators': 200}
Best F1 score: 1.0
Model:  SVM
Model:  GradientBoosting
Model:  XGBoost
Model:  LightGBM

Best model: RandomForest
Best F1 score: 1.0
